In [210]:
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
from geopy.adapters import AioHTTPAdapter
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
#%load_ext autotime
import geopandas as gpd
import geopy
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm.notebook import tqdm_notebook
import reverse_geocoder as rg


In [211]:
mcdw = pd.read_csv('../data/mcdw.20230530.csv')
mcdw.info()

C:\Users\Aaron\AppData\Local\Temp\ipykernel_38564\2137799343.py:1: DtypeWarning: Columns (9,13,19) have mixed types. Specify dtype option on import or set low_memory=False.
  mcdw = pd.read_csv('../data/mcdw.20230530.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907855 entries, 0 to 907854
Data columns (total 36 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   WMO ID                                             907855 non-null  int64  
 1   YEAR-MONTH                                         907855 non-null  object 
 2   LONGITUDE (deg)                                    907855 non-null  float64
 3   LATITUDE (deg)                                     907855 non-null  float64
 4   ELEVATION (m)                                      907855 non-null  float64
 5   STATION PRESSURE (0.1 mb)                          907855 non-null  int64  
 6   STATION PRESSURE SOURCE FLAG                       826655 non-null  object 
 7   MEAN SEA LEVEL PRESSURE (0.1 mb)                   825674 non-null  float64
 8   MEAN SEA LEVEL PRESSURE SOURCE FLAG                907855 non-null  object

In [212]:
#Droppping columns not relavent 
mcdw.drop(['ELEVATION (m)', 'STATION PRESSURE (0.1 mb)','STATION PRESSURE SOURCE FLAG', 'MEAN SEA LEVEL PRESSURE (0.1 mb)', 'MEAN SEA LEVEL PRESSURE SOURCE FLAG', 'MEAN SEA LEVEL PRESSURE HEIGHT FLAG', 'VAPOR PRESSURE (0.1 mb)', 'VAPOR PRESSURE SOURCE FLAG', 'VAPOR PRESSURE DEPARTURE (0.1 mb)', 'VAPOR PRESSURE DEPARTURE SOURCE FLAG'], axis=1, inplace=True)

In [213]:
#removing data with unknow coordinates
#mcdw = mcdw[(mcdw['LATITUDE (deg)'] != -9999.0000) & (mcdw['LONGITUDE (deg)'] != -9999.0000)]

In [214]:
len(mcdw['LATITUDE (deg)'])

907855

In [215]:
len(mcdw['LONGITUDE (deg)'])

907855

In [216]:
#trying to create pair of values inside the df
#mcdw['lat_long'] = mcdw['LATITUDE (deg)'] & mcdw['LONGITUDE (deg)']
#mcdw['lat_long']=mcdw.apply(lambda x:'%s,%s' % (x['LATITUDE (deg)'],x['LONGITUDE (deg)']),axis=1)
#mcdw['Coordinates'] = mcdw.apply(lambda row: (row['LATITUDE (deg)'], row['LONGITUDE (deg)']), axis=1)
#mcdw['lat_long'] = mcdw[mcdw['lat_long'].astype(str)]
#mcdw['geom'] = mcdw['LATITUDE (deg)'].map(str) + '','' + mcdw['LONGITUDE (deg)'].map(str)
mcdw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907855 entries, 0 to 907854
Data columns (total 26 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   WMO ID                                             907855 non-null  int64  
 1   YEAR-MONTH                                         907855 non-null  object 
 2   LONGITUDE (deg)                                    907855 non-null  float64
 3   LATITUDE (deg)                                     907855 non-null  float64
 4   MINIMUM TEMPERATURE (deg C)                        907855 non-null  int64  
 5   MINIMUM TEMPERATURE SOURCE FLAG                    605950 non-null  object 
 6   MEAN TEMPERATURE (deg C)                           907855 non-null  int64  
 7   MEAN TEMPERATURE SOURCE FLAG                       892136 non-null  object 
 8   MEAN TEMPERATURE DEPARTURE (deg C)                 907855 non-null  int64 

In [217]:
#mcdw['lat_long'] = mcdw['lat_long'].astype(str)

In [218]:
#other option of creating tuple list for the geolocator
lat = mcdw['LATITUDE (deg)']
lon = mcdw['LONGITUDE (deg)']

result = []
for i in range(len(lat)):
     #Adding to result based on indexes
     result.append((lat[i], lon[i]))

In [219]:
result

[(-9999.0, -9999.0),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8

In [220]:
#Need to group into regions These are lots of ways I did not get to work
#lat = mcdw['LATITUDE (deg)']
#lang = mcdw['LONGITUDE (deg)']
#location = geolocator.reverse("{}, {}".format(mcdw['LATITUDE (deg)'], mcdw['LONGITUDE (deg)']))

#geolocator = Nominatim(user_agent="google")
#geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
#location = geolocator.reverse(mcdw['lat_long'])
 #df['location'] = df['name'].apply(geocode)
#mcdw["location"] = mcdw['lat_long'].apply(lambda el: geolocator.reverse(el).raw["address"])
#mcdw["city"] = list(map(lambda el: geolocator.reverse(el).raw["address"], mcdw["lat_long"]))
#mcdw["city"] = list(map(lambda el: geolocator.reverse(el).raw["address"], result))

#mcdw['location'] = geolocator.reverse(f'{lat}, {lang}')

In [221]:
#locator = Nominatim(user_agent='google', timeout=10)
#rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)

In [222]:
#type(result[0])
#location = locator.reverse(result[0])
#location.raw

In [223]:
#So that made a nice little dic of locations... 

location_dic = rg.search(result)

In [224]:
len(location_dic)

907855

In [225]:
#turning dictionary into df so as to merge back in with the mcddw df
location_df = pd.DataFrame(location_dic)
location_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907855 entries, 0 to 907854
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   lat     907855 non-null  object
 1   lon     907855 non-null  object
 2   name    907855 non-null  object
 3   admin1  907855 non-null  object
 4   admin2  907855 non-null  object
 5   cc      907855 non-null  object
dtypes: object(6)
memory usage: 41.6+ MB


In [226]:
#Formatting for merge
location_df['lat'] = location_df['lat'].astype(float)
location_df['lon'] = location_df['lon'].astype(float)

In [227]:
#testing country and state name.
US = location_df[location_df['cc'] == 'US']
US.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93236 entries, 541786 to 828569
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lat     93236 non-null  float64
 1   lon     93236 non-null  float64
 2   name    93236 non-null  object 
 3   admin1  93236 non-null  object 
 4   admin2  93236 non-null  object 
 5   cc      93236 non-null  object 
dtypes: float64(2), object(4)
memory usage: 5.0+ MB


In [230]:
merged_df = pd.merge(US, mcdw, left_on=['lat', 'lon'], right_on=['LATITUDE (deg)', 'LONGITUDE (deg)'])
merged_df[merged_df['cc'] == 'US']

,lat,lon,name,admin1,admin2,cc,WMO ID,YEAR-MONTH,LONGITUDE (deg),LATITUDE (deg),...,TOTAL PRECIPITATION DEPARTURE (mm),TOTAL PRECIPITATION DEPARTURE SOURCE FLAG,PRECIPITATION QUINTILE,PRECIPITATION QUINTILE SOURCE FLAG,SUNSHINE DURATION (hours),SUNSHINE DURATION SOURCE FLAG,SUNSHINE DURATION AS LONG TERM NORMAL (%),SUNSHINE DURATION AS LONG TERM NORMAL SOURCE FLAG,VERSION,STATION NAME


In [240]:
mcdw['lat'] = mcdw['LATITUDE (deg)']
mcdw['lon'] = mcdw['LONGITUDE (deg)']
merged_df = mcdw.merge(US, how = 'inner', on = ['lat', 'lon'])
merged_df.head()

,WMO ID,YEAR-MONTH,LONGITUDE (deg),LATITUDE (deg),MINIMUM TEMPERATURE (deg C),MINIMUM TEMPERATURE SOURCE FLAG,MEAN TEMPERATURE (deg C),MEAN TEMPERATURE SOURCE FLAG,MEAN TEMPERATURE DEPARTURE (deg C),MEAN TEMPERATURE DEPARTURE SOURCE FLAG,...,SUNSHINE DURATION AS LONG TERM NORMAL (%),SUNSHINE DURATION AS LONG TERM NORMAL SOURCE FLAG,VERSION,STATION NAME,lat,lon,name,admin1,admin2,cc


In [239]:
left_merged = pd.merge(
...     US, mcdw, how="left", on=["lat", "lon"])

merged_df.head()

,WMO ID,YEAR-MONTH,LONGITUDE (deg),LATITUDE (deg),MINIMUM TEMPERATURE (deg C),MINIMUM TEMPERATURE SOURCE FLAG,MEAN TEMPERATURE (deg C),MEAN TEMPERATURE SOURCE FLAG,MEAN TEMPERATURE DEPARTURE (deg C),MEAN TEMPERATURE DEPARTURE SOURCE FLAG,...,SUNSHINE DURATION AS LONG TERM NORMAL (%),SUNSHINE DURATION AS LONG TERM NORMAL SOURCE FLAG,VERSION,STATION NAME,lat,lon,name,admin1,admin2,cc


In [242]:
inner_joined_total = US.join(
...     mcdw.set_index(["lat", "lon"]),
...     on=["lat", "lon"],
...     how="inner",
...     lsuffix="_x",
...     rsuffix="_y",)

In [243]:
inner_joined_total

,lat,lon,name,admin1,admin2,cc,WMO ID,YEAR-MONTH,LONGITUDE (deg),LATITUDE (deg),...,TOTAL PRECIPITATION DEPARTURE (mm),TOTAL PRECIPITATION DEPARTURE SOURCE FLAG,PRECIPITATION QUINTILE,PRECIPITATION QUINTILE SOURCE FLAG,SUNSHINE DURATION (hours),SUNSHINE DURATION SOURCE FLAG,SUNSHINE DURATION AS LONG TERM NORMAL (%),SUNSHINE DURATION AS LONG TERM NORMAL SOURCE FLAG,VERSION,STATION NAME
